In [46]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from sklearn.metrics.pairwise import cosine_similarity
from jikanpy import Jikan

In [47]:
animeScores = pd.read_csv('animelists_cleaned.csv', usecols = ['username', 'anime_id', 'my_score'])
animeScores.head()

,username,anime_id,my_score
0,karthiga,21,9
1,karthiga,59,7
2,karthiga,74,7
3,karthiga,120,7
4,karthiga,178,7


In [48]:
nUsers = animeScores['username'].nunique()
nAnime = animeScores['anime_id'].max()
nCols = 31284030
nFeatures = nUsers+nAnime

In [49]:
print (nUsers, nAnime)

108709 37896


In [53]:
def createPivotSparse(pdDF):
    d = {}
    a = 0
    X = sparse.lil_matrix((nUsers,nAnime)).astype('float32')
    userId = 0
    for x in pdDF.itertuples():
        if i > 1000000: break
        if x[1] not in d:
            d[x[1]] = a
            userId = a
            a = a +1
        else:
            userId = d[x[1]]
        X[userId, x[2]] = x[3]
    return X
        

In [54]:
X = sparse.csr_matrix(createPivotSparse(animeScores))

NameError: name 'i' is not defined

In [ ]:
def scaleScores(X):
    for i in range(nUsers):
        nZ = X[i].nonzero()[1]
        L = []
        for j in nZ:
            L.append(X[i,j])
        m = np.mean(L)
        s = np.std(L)
        if s == 0: s = 1
        for j in nZ:
            X[i,j] = (X[i,j]-m)/s
    return X

In [ ]:
scaledX = scaleScores(X)

In [ ]:
itemSim = cosine_similarity(scaledX.transpose(), dense_output = False)

In [ ]:
def getUserMtrx(username):
    jk = Jikan()
    userList = jk.user(username = username, request = 'animelist')['anime']
    numShows = len(userList)
    userMtrx = sparse.lil_matrix((1,nAnime))
    L = []
    for show in userList:
        L.append(show['score'])
    m = np.mean(L)
    s = np.std(L)
    for show in userList:
        userMtrx[0, show['mal_id']] = (show['score']-m)/s
                
    return sparse.csr_matrix(userMtrx)

In [ ]:
me = getUserMtrx('dakness989')

In [ ]:
mySim = cosine_similarity(scaledX, me, dense_output = False)

In [ ]:
def mostSimilarUsers(X, n=5):
    result = []
    L = []
    for i in range(n):
        j = X.argmax()
        result.append(j)
        L.append((j, X[j,0]))
        X[j,0] = -1
        
    for (i, k) in L:
        X[i,0] = k
    
    return result

In [ ]:
mySimilar = mostSimilarUsers(mySim)

In [ ]:
def AnimeRecommend(users, X):
    jk = Jikan()
    L = []
    for user in users:
        L.append(X[user].argmax())
        
    result = []
    for x in L:
        
        animeName = (jk.anime(x))['title_english']
        if animeName not in result:
            result.append(animeName)
    return result

In [ ]:
myAnime = AnimeRecommend(mySimilar, scaledX)
print(myAnime)

In [ ]:
def animeRecommendFull(username):
    userList = getUserMtrx(username)
    userSim = cosine_similarity(scaledX, userList, dense_output = False)
    similarUsers = mostSimilarUsers(userSim)
    return AnimeRecommend(similarUsers, scaledX)

In [ ]:
print(animeRecommendFull('dakness989'))